# Final Project!!!
## Crimson Dance
### Project Title?: Slay the Charts like Beyoncé

Hi team, let's do this. Please read the comments :)

<img src="http://static.celebuzz.com/uploads/2013/09/03/beyonce-33.gif" style="width:100%; align:left">


In [137]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import re
import math
import urllib2
import json

In [2]:
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import requests

## Retrieving the Songs

We want to obtain a list of songs to query information for using the Echo Nest API later.

In [227]:
# my code
# TO DO: choose the years

years = range(1995, 2015, 1)
all_links = []

for index in range(len(years)):
    link = requests.get('http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_ %(year)4d' % \
        {"year": years[index]}) 
    all_links.append(link)
    time.sleep(1)
    
yearstext = dict(zip(years, all_links))

In [228]:
# From HW1
"""
Function
--------
parse_year

Inputs
------
year: the year you want the singles for
ytextdixt: a dictionary with keys as integer years and values the downloaded web pages 
    from wikipedia for that year.
   
Returns
-------

a list of dictionaries, each of which corresponds to a single and has the
following data:

Eg:

{'band_singer': ['Brandy', 'Monica'],
  'ranking': 2,
  'song': ['The Boy Is Mine'],
  'songurl': ['/wiki/The_Boy_Is_Mine_(song)'],
  'titletext': '" The Boy Is Mine "',
  'url': ['/wiki/Brandy_Norwood', '/wiki/Monica_(entertainer)']}
  
A dictionary with the following data:
    band_singer: a list of bands/singers who made this single
    song: a list of the titles of songs on this single
    songurl: a list of the same size as song which has urls for the songs on the single 
        (see point 3 above)
    ranking: ranking of the single
    titletext: the contents of the table cell
    band_singer: a list of bands or singers on this single
    url: a list of wikipedia singer/band urls on this single: only put in the part 
        of the url from /wiki onwards
    

Notes
-----
See description and example above.
"""
# my code

def cleaner(r):
    url = []
    songurl = []
    band_singer = []
    song_name = []
    songs = r[1].findAll("a")
    singers = r[2].findAll("a")
    for song in songs:
        songurl.append(song["href"])
        song_name.append(song.get_text())
    for singer in singers:
        url.append(singer["href"])
        band_singer.append(singer.get_text())
    return [int(r[0].get_text()), r[1].get_text(), band_singer, url, 
            song_name, songurl]
    
fields = ["ranking", "titletext", "band_singer", "url", "song", "songurl"]

def parse_year(year, ytextdixt):
    page_info = ytextdixt.get(year)
    parsed = BeautifulSoup(page_info.text, "html.parser")
    rows = parsed.find("table", attrs={"class": "wikitable"}).find_all("tr")[1:]
    songs = [dict(zip(fields, cleaner(row.find_all("th") + row.find_all("td")))) for row in rows]
    return songs

In [229]:
yearinfo = {y:parse_year(y, yearstext) for y in years}

In [230]:
# fd = open("tempdata/yearinfo.json","w")
# json.dump(yearinfo, fd)
# fd.close()

In [231]:
# del yearinfo
with open("tempdata/yearinfo.json", "r") as fd:
    yearinfo = json.load(fd)

In [232]:
# From HW1
yeardict={}
for y in yearinfo.keys():
    yearlist=yearinfo[y]
    yearlist2=[]
    for idict in yearlist:
        singers=idict['band_singer']
        for i,s in enumerate(singers):
            songs=idict['song']
            for j,so in enumerate(songs):#now inside each singer song combination
                nd={}
                nd['band_singer']=s
                nd['url']=idict['url'][i]
                nd['song']=so
                nd['songurl']=idict['songurl'][j]
                nd['ranking']=idict['ranking']
                yearlist2.append(nd)
    yeardict[y]=pd.DataFrame(yearlist2)#one for each year
yearspanel=pd.Panel.from_dict(yeardict, orient="minor")#stack dataframes into a panel
hierframe=yearspanel.to_frame() #flattening leads to a hierarchical index

In [233]:
flatframe = hierframe.reset_index()
flatframe.head()

,major,minor,band_singer,ranking,song,songurl,url
0,0,1995,Coolio,1,Gangsta's Paradise,/wiki/Gangsta%27s_Paradise_(song),/wiki/Coolio
1,0,1996,Los del Río,1,Macarena,/wiki/Macarena_(song),/wiki/Los_del_R%C3%ADo
2,0,1997,Elton John,1,Something About the Way You Look Tonight,/wiki/Something_About_the_Way_You_Look_Tonight,/wiki/Elton_John
3,0,1998,Next,1,Too Close,/wiki/Too_Close_(Next_song),/wiki/Next_(group)
4,0,1999,Cher,1,Believe,/wiki/Believe_(Cher_song),/wiki/Cher


In [234]:
flatframe = flatframe.rename(columns={'minor':'year'})
del flatframe['major']
flatframe.head()

,year,band_singer,ranking,song,songurl,url
0,1995,Coolio,1,Gangsta's Paradise,/wiki/Gangsta%27s_Paradise_(song),/wiki/Coolio
1,1996,Los del Río,1,Macarena,/wiki/Macarena_(song),/wiki/Los_del_R%C3%ADo
2,1997,Elton John,1,Something About the Way You Look Tonight,/wiki/Something_About_the_Way_You_Look_Tonight,/wiki/Elton_John
3,1998,Next,1,Too Close,/wiki/Too_Close_(Next_song),/wiki/Next_(group)
4,1999,Cher,1,Believe,/wiki/Believe_(Cher_song),/wiki/Cher


In [235]:
flatframe.sort(["year", "ranking"], inplace=True)
flatframe.head()

,year,band_singer,ranking,song,songurl,url
0,1995,Coolio,1,Gangsta's Paradise,/wiki/Gangsta%27s_Paradise_(song),/wiki/Coolio
20,1995,L.V.,1,Gangsta's Paradise,/wiki/Gangsta%27s_Paradise_(song),/wiki/L.V._(singer)
40,1995,TLC,2,Waterfalls,/wiki/Waterfalls_(TLC_song),/wiki/TLC_(band)
60,1995,TLC,3,Creep,/wiki/Creep_(TLC_song),/wiki/TLC_(band)
80,1995,Seal,4,Kiss from a Rose,/wiki/Kiss_from_a_Rose,/wiki/Seal_(musician)


In [236]:
len(flatframe.url)

2641

## Retrieving the Data for the Songs

I tried retrieving data for 10 years worth of songs, however the API yelled at me; I think went over the maximum number of requests. With my API key, I can request something every three seconds (hence `time.sleep(3)`). I think each of us should get a key so we can do the requests together :)

Here, I am retrieving data for only one year; the requests should take less than 10 minutes. I think we can perhaps do five years at a time.

In [237]:
# change year into integer
flatframe["year"] = flatframe["year"].astype(int)

In [239]:
# 2005 was used as a test
y2005 = flatframe[flatframe.year==2005]
y2005.head(6)

,year,band_singer,ranking,song,songurl,url
10,2005,Mariah Carey,1,We Belong Together,/wiki/We_Belong_Together_(Mariah_Carey_song),/wiki/Mariah_Carey
30,2005,Gwen Stefani,2,Hollaback Girl,/wiki/Hollaback_Girl,/wiki/Gwen_Stefani
50,2005,Mario,3,Let Me Love You,/wiki/Let_Me_Love_You_(Mario_song),/wiki/Mario_(entertainer)
70,2005,Kelly Clarkson,4,Since U Been Gone,/wiki/Since_U_Been_Gone,/wiki/Kelly_Clarkson
90,2005,Ciara,5,"1, 2 Step","/wiki/1,_2_Step",/wiki/Ciara
110,2005,Missy Elliott,5,"1, 2 Step","/wiki/1,_2_Step",/wiki/Missy_Elliott


In [240]:
len(y2005) ## over 100, due to featurings

138

In [241]:
# FUNCTION get_songs

def get_songs(year_start, year_end):
    all_songs = []
    featurings = []
    billboard = flatframe[(flatframe.year >= year_start) & (flatframe.year <= year_end)]
    for index, row in billboard.iterrows():
        song = urllib2.quote(row["song"].encode("utf-8"))
        artist = urllib2.quote(row["band_singer"].encode("utf-8"))
        link = requests.get('http://developer.echonest.com/api/v4/song/search?api_key=GT3HIFAMRWWCOWIAZ' + 
                            '&bucket=artist_discovery&bucket=artist_discovery_rank&bucket=artist_familiarity' +
                            '&bucket=artist_familiarity_rank&bucket=artist_hotttnesss&bucket=artist_hotttnesss_rank' +
                            '&bucket=artist_location&bucket=song_currency&bucket=song_currency_rank' +
                            '&bucket=song_discovery&bucket=song_discovery_rank&bucket=song_hotttnesss'+
                            '&bucket=song_hotttnesss_rank&bucket=song_type&bucket=audio_summary' +
                            '&results=1&title=%(song)s&artist=%(artist)s' % \
                            {"song": song, "artist": artist})
        result_json = json.loads(link.text)
        if not result_json["response"]["songs"]:
            featurings.append(index)
        all_songs.append(result_json)
        time.sleep(3)
    return all_songs
    
## echonest does not seem to account for featurings, see below:
## https://developer.echonest.com/forums/thread/2007

In [243]:
# %% time
# all_songs_95_04 = get_songs(1995, 2004)

In [185]:
## for some reason, echonest was not allowing us to do too many requests at once, 
## so the songs from 2005-2014 were requested in parts
## all_songs_06_14 = all_songs_06_partial[:142] + all_songs_06_partial2 + all_songs_06_partial3

In [25]:
# fd=open("tempdata/all_songs_95_04.json","w")
# json.dump(all_songs_95_04, fd)
# fd.close()

In [26]:
# If you need to reload you can use this code
# with open("tempdata/allsongs05.json") as json_file:
#     all_songs_95_04 = json.load(json_file)
# you might have to add code to get the correct types..

In [197]:
columns = ["title", "id", "artist_name", "artist_id", "audio_summary", "artist_discovery",
           "artist_discovery_rank", "artist_familiarity", "artist_familiarity_rank",
           "artist_hotttnesss", "artist_hotttnesss_rank", "artist_location", "song_currency", 
           "song_currency_rank", "song_discovery", "song_discovery_rank", "song_hotttnesss", 
           "song_hotttnesss_rank", "song_type"]

audio_summary = ["acousticness", "analysis_url","audio_md5", "audio_md5", "duration", "energy", "key", 
                 "liveness", "loudness","mode", "speechiness", "tempo", "time_signature", "valence"]

In [ ]:
# FUNCTION flatten

def flatten(all_songs, billboard):
    print len(all_songs), len(billboard) ## should be equal
    songdicts=[]
    fails = []
    total_feature_fails=0
    
    for a, i in zip(all_songs, range(len(all_songs))):
        d={}
        if a["response"]["status"]["message"] == "Success":

            if  not a["response"]["songs"]:
                total_feature_fails=total_feature_fails+1
            for b in a["response"]["songs"]:
                for var in set(b.keys()).intersection(columns):
                    if var == "audio_summary":
                        for v in set(b[var].keys()).intersection(audio_summary):
                            d[v] = b[var][v]
                    d[var] = b[var]
                    d["index"] = billboard.index[i]

                songdicts.append(d)  

        # if the requests failed
        else:
            fails.append(i)
            print i, a
    return pd.DataFrame(songdicts)

In [244]:
# df_echonest = flatten(all_songs_95_04, flatframe[(flatframe.year >= 1995) & (flatframe.year <= 2004)])
# df_echonest.head(6)

In [143]:
# adding in the old dataframe so that we have ranking information
# all_95_04=pd.merge(df_echonest, flatframe, left_on=["index"], right_index=True)
# all_95_04.head()

In [146]:
# all_95_04.to_csv("tempdata/all_95_04.csv", encoding='utf-8')

In [206]:
# frames = [all_05, all_06_14]
# all_05_14 = pd.concat(frames)

In [208]:
# all_05_14.to_csv("tempdata/all_05_14.csv", encoding='utf-8')

In [64]:
# TO DO:
# more years!!!

## Exploratory Data Analysis

In [ ]:
# BELOW IS EXTRA

In [194]:
%%time
# my code

all_songs_0614 = []
featurings = []

for index, row in y2006_2014.iterrows():
    song = urllib2.quote(row["song"].encode("utf-8"))
    artist = urllib2.quote(row["band_singer"].encode("utf-8"))
    link = requests.get('http://developer.echonest.com/api/v4/song/search?api_key=GT3HIFAMRWWCOWIAZ' + 
                        '&bucket=artist_discovery&bucket=artist_discovery_rank&bucket=artist_familiarity' +
                        '&bucket=artist_familiarity_rank&bucket=artist_hotttnesss&bucket=artist_hotttnesss_rank' +
                        '&bucket=artist_location&bucket=song_currency&bucket=song_currency_rank' +
                        '&bucket=song_discovery&bucket=song_discovery_rank&bucket=song_hotttnesss'+
                        '&bucket=song_hotttnesss_rank&bucket=song_type&bucket=audio_summary' +
                        '&results=1&title=%(song)s&artist=%(artist)s' % \
                        {"song": song, "artist": artist})
    result_json = json.loads(link.text)
    if not result_json["response"]["songs"]:
        featurings.append(index)
        print index, song, artist
    all_songs_0614.append(result_json)
    time.sleep(3)
    
## echonest does not seem to account for featurings, see below:
## https://developer.echonest.com/forums/thread/2007

31 Promiscuous Timbaland
61 Hips%20Don%27t%20Lie Wyclef%20Jean
121 SexyBack Timbaland
181 Grillz Ali%20%26%20Gipp
221 Buttons Snoop%20Dogg
241 Run%20It%21 Juelz%20Santana
271 It%27s%20Goin%27%20Down Nitti
341 Snap%20Yo%20Fingers E-40
351 Snap%20Yo%20Fingers Sean%20Paul
481 Money%20Maker Pharrell
501 Ms.%20New%20Booty Ying%20Yang%20Twins
521 %28When%20You%20Gonna%29%20Give%20It%20Up%20to%20Me Keyshia%20Cole
571 I%27m%20%27n%20Luv%20%28Wit%20a%20Stripper%29 Mike%20Jones
601 Where%27d%20You%20Go Holly%20Brook
611 Where%27d%20You%20Go Jonah%20Matranga
711 U%20and%20Dat T-Pain
721 U%20and%20Dat Kandi%20Girl
761 So%20What Ciara
781 Do%20It%20to%20It Sean%20P
841 Unpredictable Ludacris
901 Pullin%27%20Me%20Back Tyrese
951 Smack%20That Eminem
991 Gimme%20That Lil%20Wayne
1021 Who%20Says%20You%20Can%27t%20Go%20Home Jennifer%20Nettles
1121 Show%20Stopper Yung%20Joc
1191 Beep will.i.am
1211 I%20Think%20They%20Like%20Me Jermaine%20Dupri
1271 Soul%20Survivor Akon
1326 Stay%20Fly 8Ball%20%26%20MJG
4

In [195]:
len(all_songs_0614)

1274

In [ ]:
# Below is the code I had before... which can be replaced by the above lol

#         d["title"] = b["title"]
#         d["id"] = b["id"]
#         d["artist_name"] = b["artist_name"]
#         d["artist_id"] = b["artist_id"]
#         d["acousticness"] = b["audio_summary"]["acousticness"]
#         d["analysis_url"] = b["audio_summary"]["analysis_url"]
#         d["audio_md5"] = b["audio_summary"]["audio_md5"]
#         d["duration"] = b["audio_summary"]["duration"]
#         d["energy"] = b["audio_summary"]["energy"]
#         d["key"] = b["audio_summary"]["key"]
#         d["liveness"] = b["audio_summary"]["liveness"]
#         d["loudness"] = b["audio_summary"]["loudness"]
#         d["mode"] = b["audio_summary"]["mode"]
#         d["speechiness"] = b["audio_summary"]["speechiness"]
#         d["tempo"] = b["audio_summary"]["tempo"]
#         d["time_signature"] = b["audio_summary"]["time_signature"]
#         d["valence"] = b["audio_summary"]["valence"]
        
#         d["artist_discovery"] = b["artist_discovery"]
        
#         if b["artist_discovery"]==0:
#             d["artist_discovery_rank"] = "nan"
#         else:
#             d["artist_discovery_rank"] = b["artist_discovery_rank"]
            
#         d["artist_familiarity"] = b["artist_familiarity"]
#         if b["artist_familiarity"]==0:
#             d["artist_familiarity_rank"] = "nan"
#         else:
#             d["artist_familiarity_rank"] = b["artist_familiarity_rank"]
        
#         d["artist_hotttnesss"] = b["artist_hotttnesss"]
#         if b["artist_hotttnesss"]==0:
#             d["artist_hotttnesss_rank"] = "nan"
#         else:
#             d["artist_hotttnesss_rank"] = b["artist_hotttnesss_rank"]

#         d["artist_location"] = b["artist_location"]
        
#         d["song_currency"] = b["song_currency"]
#         if b["song_currency"]==0:
#             d["song_currency_rank"] = "nan"
#         else:
#             d["song_currency_rank"] = b["song_currency_rank"]

#         d["song_discovery"] = b["song_discovery"]
#         if b["song_discovery"]==0:
#             d["song_discovery_rank"] = "nan"
#         else:
#             d["song_discovery_rank"] = b["song_discovery_rank"]

#         d["song_hotttnesss"] = b["song_hotttnesss"]
#         if b["song_hotttnesss"]==0:
#             d["song_hotttnesss_rank"] = "nan"
#         else:
#             d["song_hotttnesss_rank"] = b["song_hotttnesss_rank"]

#         d["song_type"] = b["song_type"]